In [1]:
! pip install beautifulsoup4 requests



[notice] A new release of pip is available: 23.1.2 -> 24.0
[notice] To update, run: python.exe -m pip install --upgrade pip


In [85]:
import pandas as pd
import requests
from bs4 import BeautifulSoup

In [86]:
import requests

url = "https://myanimelist.net/topanime.php"
response = requests.get(url)
#html_content = response.text



# Check if request was successful
if response.status_code == 200:
    # Parse the content
    soup = BeautifulSoup(response.content, 'html.parser')
    # Now you can work with the parsed content (soup object)
    print("done")
else:
    print("Error:", response.status_code)

done


In [216]:
#soup = BeautifulSoup(response, "html.parser")

In [87]:
anime_names =[]
anime_details = []
scores = []
links = []
descriptions = []
english_title= []

In [88]:

# Base URL of the page
base_url = 'https://myanimelist.net/topanime.php'


for page in range(17, 19):  # You can adjust the range to scrape more pages
    # Construct the URL for the current page
    url = f"{base_url}?limit={page * 50}"  # MyAnimeList paginates every 50 entries
    print(f"Scraping URL: {url}")
    response = requests.get(url).text
    soup = BeautifulSoup(response, "html.parser")
    
    # Find all anime elements in the table rows
    anime_elements = soup.find_all("tr", class_="ranking-list")
    
    for anime_element in anime_elements:
        # Get the title
         # Get the title and link
        title_element = anime_element.find("h3", class_="anime_ranking_h3").find("a")
        link = title_element.get("href") if title_element else None
        title = title_element.text.strip() if title_element else "Title not found"
            
        
        anime_d = anime_element.find("div", class_="information di-ib mt4")
        score = anime_element.find("div", class_="js-top-ranking-score-col di-ib al")
        
        # Extract title if available
        if title_element:
            anime_names.append(title_element.text.strip())
        
        # Extract anime details if available
        if anime_d:
            anime_details.append(anime_d.get_text(separator="\n").strip())
        
        if link:
            links.append(link)
        
        # Extract score if available
        if score:
            scores.append(score.text.strip())
    



Scraping URL: https://myanimelist.net/topanime.php?limit=850
Scraping URL: https://myanimelist.net/topanime.php?limit=900


In [89]:
for link in links:
        print(f"Fetching description for: - {link}")

        # Send request to the specific anime page
        response1 = requests.get(link).text
        soup1 = BeautifulSoup(response1, "html.parser")

        # Try to find the description
        description_tag = soup1.find("p", itemprop="description")
        
        english = soup1.find("div",  class_="js-alternative-titles hide")
        if description_tag:
            description = description_tag.text.strip()
            descriptions.append(description)
        else:
            descriptions.append("NAN")
        if english:
            english_t = english.find("span", class_="dark_text").find_next_sibling(text=True)
            if english_t:
                english_title.append(english_t)
        else:
            english_title.append("NAN")

Fetching description for: - https://myanimelist.net/anime/14719/JoJo_no_Kimyou_na_Bouken_TV
Fetching description for: - https://myanimelist.net/anime/3572/Macross_F
Fetching description for: - https://myanimelist.net/anime/36563/Megalo_Box
Fetching description for: - https://myanimelist.net/anime/780/Meitantei_Conan_Movie_02__14-banme_no_Target
Fetching description for: - https://myanimelist.net/anime/31490/One_Piece_Film__Gold
Fetching description for: - https://myanimelist.net/anime/38201/Osomatsu-san_Movie
Fetching description for: - https://myanimelist.net/anime/957/Saiunkoku_Monogatari
Fetching description for: - https://myanimelist.net/anime/1594/Jigoku_Shoujo_Futakomori
Fetching description for: - https://myanimelist.net/anime/52168/Kidou_Senshi_Gundam__Suisei_no_Majo_-_Prologue
Fetching description for: - https://myanimelist.net/anime/16782/Kotonoha_no_Niwa
Fetching description for: - https://myanimelist.net/anime/55855/Kuroshitsuji__Kishuku_Gakkou-hen
Fetching description for:

Fetching description for: - https://myanimelist.net/anime/8408/Durarara_Specials
Fetching description for: - https://myanimelist.net/anime/24471/Hoozuki_no_Reitetsu_OVA
Fetching description for: - https://myanimelist.net/anime/48548/5-toubun_no_Hanayome_Movie
Fetching description for: - https://myanimelist.net/anime/32998/91_Days
Fetching description for: - https://myanimelist.net/anime/49785/Fairy_Tail__100-nen_Quest
Fetching description for: - https://myanimelist.net/anime/40262/Haikyuu_Riku_vs_Kuu
Fetching description for: - https://myanimelist.net/anime/14175/Hanasaku_Iroha_Movie__Home_Sweet_Home
Fetching description for: - https://myanimelist.net/anime/3604/Hidamari_Sketch_x_365
Fetching description for: - https://myanimelist.net/anime/36593/Hug_tto_Precure
Fetching description for: - https://myanimelist.net/anime/39534/Jibaku_Shounen_Hanako-kun


In [90]:
len(descriptions)

100

In [91]:
print(len(anime_names))
print(len(scores))
print(len(english_title))

100
100
100


In [92]:
anime = pd.DataFrame({
    "Title": anime_names, 
    "anime_details": anime_details,
    "score": scores,
    "description" : descriptions,
    "English_titles" : english_title
})

In [93]:
anime

,Title,anime_details,score,description,English_titles
0,JoJo no Kimyou na Bouken (TV),TV (26 eps)\n\n Oct 2012 - Apr 2013\n\n...,7.88,The year is 1868; English nobleman George Joes...,JoJo's Bizarre Adventure (2012)\n
1,Macross F,TV (25 eps)\n\n Apr 2008 - Sep 2008\n\n...,7.88,Following a catastrophic war against a race of...,Macross Frontier\n
2,Megalo Box,TV (13 eps)\n\n Apr 2018 - Jun 2018\n\n...,7.88,"""To be quiet and do as you're told, that's the...",Megalobox\n
3,Meitantei Conan Movie 02: 14-banme no Target,Movie (1 eps)\n\n Apr 1998 - Apr 1998\n...,7.88,A mysterious attacker has appeared and is assa...,Case Closed Movie 02: The Fourteenth Target\n...
4,One Piece Film: Gold,Movie (1 eps)\n\n Jul 2016 - Jul 2016\n...,7.88,Monkey D. Luffy and his Straw Hat Crew have fi...,One Piece Film 13: Gold\n
...,...,...,...,...,...
95,Haikyuu!! Riku vs. Kuu,OVA (2 eps)\n\n Jan 2020 - Jan 2020\n\n...,7.83,An intense battle rages on at the Tokyo Qualif...,Haikyu!! Land vs. Air\n
96,Hanasaku Iroha Movie: Home Sweet Home,Movie (1 eps)\n\n Mar 2013 - Mar 2013\n...,7.83,Ohana Matsumae has been working at Kissui Inn ...,Hanasaku Iroha the Movie: Home Sweet Home\n ...
97,Hidamari Sketch x 365,TV (13 eps)\n\n Jul 2008 - Sep 2008\n\n...,7.83,"After passing her entrance exam, Yuno enrolls ...",Hidamari Sketch x 365\n
98,Hug tto! Precure,TV (49 eps)\n\n Feb 2018 - Jan 2019\n\n...,7.83,"It's her first day at a new school, and the ch...",NAN


In [94]:
anime["description"].isnull().sum()

0

In [95]:
#df = pd.DataFrame(anime)

# Export DataFrame to CSV file
anime.to_csv('anime15_list.csv', index=False)  # Set index=False to exclude row indices from the CSV file

# Gemini 

INFO: pip is looking at multiple versions of grpcio-status to determine which version is compatible with other requirements. This could take a while.
INFO: pip is still looking at multiple versions of grpcio-status to determine which version is compatible with other requirements. This could take a while.
INFO: This is taking longer than usual. You might need to provide the dependency resolver with stricter constraints to reduce runtime. See https://pip.pypa.io/warnings/backtracking for guidance. If you want to abort this run, press Ctrl + C.
   ---------------------------------------- 117.3/117.3 kB 1.4 MB/s eta 0:00:00
   ---------------------------------------- 109.3/109.3 kB 6.6 MB/s eta 0:00:00
   ---------------------------------------- 156.5/156.5 kB 9.1 MB/s eta 0:00:00
   ---------------------------------------- 50.1/50.1 kB 2.5 MB/s eta 0:00:00
   ---------------------------------------- 220.9/220.9 kB 3.4 MB/s eta 0:00:00
   ---------------------------------------- 413.4/413.

DEPRECATION: pyodbc 4.0.0-unsupported has a non-standard version number. pip 24.1 will enforce this behaviour change. A possible replacement is to upgrade to a newer version of pyodbc or contact the author to suggest that they release a version with a conforming version number. Discussion can be found at https://github.com/pypa/pip/issues/12063
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
mediapipe 0.10.0 requires protobuf<4,>=3.11, but you have protobuf 4.25.5 which is incompatible.

[notice] A new release of pip is available: 24.0 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip



  Attempting uninstall: protobuf
    Found existing installation: protobuf 3.20.3
    Uninstalling protobuf-3.20.3:
      Successfully uninstalled protobuf-3.20.3
  Attempting uninstall: grpcio
    Found existing installation: grpcio 1.53.0
    Uninstalling grpcio-1.53.0:
      Successfully uninstalled grpcio-1.53.0


DEPRECATION: pyodbc 4.0.0-unsupported has a non-standard version number. pip 24.1 will enforce this behaviour change. A possible replacement is to upgrade to a newer version of pyodbc or contact the author to suggest that they release a version with a conforming version number. Discussion can be found at https://github.com/pypa/pip/issues/12063

[notice] A new release of pip is available: 24.0 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


  0%|                                                                                            | 0/1 [00:00<?, ?it/s]

Error processing Sousou no Frieren: 404 Requested entity was not found.
Error processing One Piece Fan Letter: 404 Requested entity was not found.
Error processing Fullmetal Alchemist: Brotherhood: 404 Requested entity was not found.
Error processing Steins;Gate: 404 Requested entity was not found.
Error processing Shingeki no Kyojin Season 3 Part 2: 404 Requested entity was not found.


100%|████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:07<00:00,  7.81s/it]

Processed 5 of 5 titles
Processing complete! Results saved to anime_descriptions_gemini.csv
